# Simple iteration for systems of linear equations

First, generate a random diagonally dominant matrix, for testing.

In [5]:
import numpy as np
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

# I.  Jacobi iteration

Given

$$
A x = b
$$

separate the diagonal part $D$,

$$ A = D + (A - D) $$

and write

$$
x = D^{-1} (D - A) x + D^{-1} b\;.
$$

Then iterate

$$
x_{n + 1} = B x_{n} + c\;,
$$

where 

$$
B = D^{-1} (A - D) \qquad \text{and} \qquad c = D^{-1} b
$$


Let's construct the matrix and the r.h.s. for the Jacobi iteration

In [6]:
diag_1d = np.diag(A)

B = -A.copy()
np.fill_diagonal(B, 0)

D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B 
c = invD @ b

In [7]:
# sanity checks
from numpy.testing import assert_allclose

assert_allclose(-B + D, A)


# xx is a "ground truth" solution, compute it using a direct method
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)

Check that $\| B\| \leqslant 1$:

In [8]:
np.linalg.norm(BB)

0.36436161983015336

### Do the Jacobi iteration

In [9]:
n_iter = 50

x0 = np.ones(n)
x = x0
for _ in range(n_iter):
    x = BB @ x + c

In [10]:
# Check the result:

A @ x - b

array([ 0.00000000e+00,  2.22044605e-16,  0.00000000e+00, -1.11022302e-16,
        0.00000000e+00,  0.00000000e+00, -2.08166817e-17,  0.00000000e+00,
        0.00000000e+00,  2.22044605e-16])

### Task I.1

Collect the proof-of-concept above into a single function implementing the Jacobi iteration. This function should receive the r.h.s. matrix $A$, the l.h.s. vector `b`, and the number of iterations to perform.


The matrix $A$ in the illustration above is strongly diagonally dominant, by construction. 
What happens if the diagonal matrix elements of $A$ are made smaller? Check the convergence of the Jacobi iteration, and check the value of the norm of $B$.

(20% of the total grade)


In [11]:
def jacobi_iter(A, b, x0, eps=1e-5, niter=10000):
    U = A.copy()
    xprev = x0
    D = np.diag(np.diag(U))
    R = A-D
    Dinv = np.diag(1 / np.diag(U))
    if max(abs(np.linalg.eig(Dinv@R)[0]))<1:
        for _ in range(niter):
            xcurr = Dinv@(b-R@xprev)
            if np.linalg.norm(xcurr-xprev) < eps:
                return xcurr
            else:
                xprev = xcurr
    else:
        raise ValueError




# II. Seidel's iteration.

##### Task II.1

Implement the Seidel's iteration. 

Test it on a random matrix. Study the convergence of iterations, relate to the norm of the iteration matrix.

(30% of the total grade)

In [15]:
# ... ENTER YOUR CODE HERE ...
def seidel(A1, b, x0, eps=1e-6, niter=10000):
    A = A1.copy()
    L = np.tril(A)
    U = A - L
    Linv = np.linalg.inv(L)
    x_prev = x0
    for _ in range(niter):
        x_curr = Linv@(b-U @ x_prev)
        if np.linalg.norm(x_curr-x_prev)<eps:
            return x_curr
        else:
            x_prev = x_curr
    print("Reached max iteration")
    return x_prev


print(A@seidel(A,b,x0)-b)

[-4.50110384e-07 -3.79819363e-07 -1.96520935e-07 -7.37035032e-08
 -5.41916456e-09  8.60068685e-08  3.72531104e-08  1.11155928e-08
  1.58400050e-08  1.11022302e-16]


# III. Minimum residual scheme

### Task III.1

Implement the $\textit{minimum residual}$ scheme: an explicit non-stationary method, where at each step you select the iteration parameter $\tau_n$ to minimize the residual $\mathbf{r}_{n+1}$ given $\mathbf{r}_n$. Test it on a random matrix, study the convergence to the solution, in terms of the norm of the residual and the deviation from the ground truth solution (which you can obtain using a direct method). Study how the iteration parameter $\tau_n$ changes as iterations progress.

(50% of the grade)

In [ ]:
# ... ENTER YOUR CODE HERE ...